# Learn OpenAI Whisper - Chapter 9
## Notebook 3: Fine-tuning voice cloning with Deep Learning Art School (DLAS)

This notebook complements the book [Learn OpenAI Whisper](https://a.co/d/1p5k4Tg).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1wnomL0dxmU9CgPKIgazR8AocolEYjAe5)

This notebook fine-tunes a voice cloning model using the DLAS toolkit. It is based on the Tortoise fine-tuning with DLAS project by James Betker (https://github.com/152334H/DL-Art-School).  The original batch size was 128 but changed to 90 since colab free tier only gives you an NVIDIA T4, a 128 batch size used 16GB of VRAM.

## 1. Checking the GPU:
The code first checks if an NVIDIA GPU is available using the `nvidia-smi` command. It prints out the GPU information if connected.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr 11 22:47:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 2. Checking virtual memory:
It then checks the available RAM on the runtime using the `psutil` library. It prints a message if using a high-RAM runtime.

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


## 3. Mounting Google Drive:
The code mounts the user's Google Drive to save trained checkpoints and load the dataset.

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## 4. Installing requirements:
It clones the DLAS repository, downloads pre-trained model checkpoints, and installs the required dependencies.

In [4]:
!git clone https://github.com/josuebatista/DL-Art-School.git
%cd DL-Art-School
!wget https://huggingface.co/Gatozu35/tortoise-tts/resolve/main/dvae.pth -O experiments/dvae.pth
!wget https://huggingface.co/jbetker/tortoise-tts-v2/resolve/main/.models/autoregressive.pth -O experiments/autoregressive.pth
!pip install -r codes/requirements.laxed.txt

Cloning into 'DL-Art-School'...
remote: Enumerating objects: 16612, done.
remote: Total 16612 (delta 0), reused 0 (delta 0), pack-reused 16612
Receiving objects: 100% (16612/16612), 12.15 MiB | 18.79 MiB/s, done.
Resolving deltas: 100% (13325/13325), done.
/content/DL-Art-School
--2024-04-11 22:56:15--  https://huggingface.co/Gatozu35/tortoise-tts/resolve/main/dvae.pth
Resolving huggingface.co (huggingface.co)... 13.35.7.81, 13.35.7.5, 13.35.7.57, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.81|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/9f/83/9f8300e5ccd418d283e72c07d1851f269dbd2ca7bae49cee21285965d9bebe14/a990825371506c16bcf0e8167bf24ccf82f65bb6a1dbcbfcf058d76f9b197e35?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27dvae.pth%3B+filename%3D%22dvae.pth%22%3B&Expires=1713135376&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMzEzNTM3Nn19LCJSZX

# MUST RESTART SESSION BEFORE PROCEEDING
The following packages were previously imported in this runtime:
  
  `[pydevd_plugins]`

You must restart the runtime in order to use newly installed versions.

Restarting will lose all runtime state, including local variables.


## Check the integrity of the dVAE checkpoint
You should see the following message when verified:

`a990825371506c16bcf0e8167bf24ccf82f65bb6a1dbcbfcf058d76f9b197e35  ../DL-Art-School/experiments/dvae.pth`


In [1]:
'''
Downgrading Transformers to previous version.
It seems that last release of transformers broke the model loader.
'''
!pip install transformers==4.29.2

# Check the integrity of the dVAE checkpoint
!sha256sum /content/DL-Art-School/experiments/dvae.pth | grep a990825371506c16bcf0e8167bf24ccf82f65bb6a1dbcbfcf058d76f9b197e35 || echo "SOMETHING IS WRONG WITH THE CHECKPOINT; REPORT THIS AS A GITHUB ISSUE AND DO NOT PROCEED"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
a990825371506c16bcf0e8167bf24ccf82f65bb6a1dbcbfcf058d76f9b197e35  /content/DL-Art-School/experiments/dvae.pth


## 5. Calculating hyperparameters:
This section automatically calculates suggested hyperparameters for training based on the provided dataset sizes. It adjusts the batch sizes to minimize leftover samples in each epoch, calculates the number of steps per epoch, and determines the frequencies for learning rate decay, validation, and checkpoint saving. Hyperparameters are crucial as they directly control the training algorithm's behavior and significantly impact the model's performance.

To find the path to `Dataset_Training_Path` and `ValidationDataset_Training_Path` click on Google Colab's `Files` and search for directory `output` where the DJ format datasets are stored from the previous step in notebook #2.

<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter09/images/ch09_3_Google_Colab_DJ_format_directory.JPG" width=600>


In [2]:
from pathlib import Path
from math import ceil
DEFAULT_TRAIN_BS = 64
DEFAULT_VAL_BS = 32
#@markdown # Hyperparameter calculation
#@markdown Run this cell to obtain suggested parameters for training
Dataset_Training_Path = "/content/gdrive/MyDrive/output/Learn_OAI_Whisper_Sample_Audio01.mp3_2024_04_11-21_39/train.txt" #@param {type:"string"}
ValidationDataset_Training_Path = "/content/gdrive/MyDrive/output/Learn_OAI_Whisper_Sample_Audio01.mp3_2024_04_11-21_39/valid.txt" #@param {type:"string"}

#@markdown ### **NOTE**: Dataset must be in the following format.

#@markdown  `dataset/`
#@markdown * ---├── `val.txt`
#@markdown * ---├── `train.txt`
#@markdown * ---├── `wavs/`

#@markdown `wavs/` directory must contain `.wav` files.

#@markdown  Example for `train.txt` and `val.txt`:

#@markdown * `wavs/A.wav|Write the transcribed audio here.`

#@markdown todo: actually check the dataset structure

if Dataset_Training_Path == ValidationDataset_Training_Path:
  print("WARNING: training dataset path == validation dataset path!!!")
  print("\tThis is technically okay but will make all of the validation metrics useless. ")
  print("it will also SUBSTANTIALLY slow down the rate of training, because validation datasets are supposed to be much smaller than training ones.")

def txt_file_lines(p: str) -> int:
  return len(Path(p).read_text().strip().split('\n'))
training_samples = txt_file_lines(Dataset_Training_Path)
val_samples = txt_file_lines(ValidationDataset_Training_Path)

if training_samples < 128: print("WARNING: very small dataset! the smallest dataset tested thus far had ~200 samples.")
if val_samples < 20: print("WARNING: very small validation dataset! val batch size will be scaled down to account")

def div_spillover(n: int, bs: int) -> int: # returns new batch size
  epoch_steps,remain = divmod(n,bs)
  if epoch_steps*2 > bs: return bs # don't bother optimising this stuff if epoch_steps are high
  if not remain: return bs # unlikely but still

  if remain*2 < bs: # "easier" to get rid of remainder -- should increase bs
    target_bs = n//epoch_steps
  else: # easier to increase epoch_steps by 1 -- decrease bs
    target_bs = n//(epoch_steps+1)
  assert n%target_bs < epoch_steps+2 # should be very few extra
  return target_bs

if training_samples < DEFAULT_TRAIN_BS:
  print("WARNING: dataset is smaller than a single batch. This will almost certainly perform poorly. Trying anyway")
  train_bs = training_samples
else:
  train_bs = div_spillover(training_samples, DEFAULT_TRAIN_BS)
if val_samples < DEFAULT_VAL_BS:
  val_bs = val_samples
else:
  val_bs = div_spillover(val_samples, DEFAULT_VAL_BS)

steps_per_epoch = training_samples//train_bs
lr_decay_epochs = [20, 40, 56, 72]
lr_decay_steps = [steps_per_epoch * e for e in lr_decay_epochs]
print_freq = min(100, max(20, steps_per_epoch))
val_freq = save_checkpoint_freq = print_freq * 3

print("===CALCULATED SETTINGS===")
print(f'{train_bs=} {val_bs=}')
print(f'{val_freq=} {lr_decay_steps=}')
print(f'{print_freq=} {save_checkpoint_freq=}')

===CALCULATED SETTINGS===
train_bs=4 val_bs=1
val_freq=60 lr_decay_steps=[20, 40, 56, 72]
print_freq=20 save_checkpoint_freq=60


## 6. Training settings:
The purpose of this section is to allow us to customize the training settings according to their requirements and available resources. It provides flexibility in naming the experiment, specifying dataset names, enabling or disabling certain features, and overriding calculated settings. The code also includes notes and warnings to guide the user in making appropriate choices based on their system's storage and computational capabilities.

In [7]:
#@markdown ##_Settings for normal users:_
Experiment_Name = "Learn_OAI_Whisper_20240411_JRB" #@param {type:"string"}
Dataset_Training_Name= "TestDataset" #@param {type:"string"}
ValidationDataset_Name = "TestValidation" # this seems to be useless??? @param {type:"string"}
SaveTrainingStates = False # @param {type:"boolean"}
Keep_Last_N_Checkpoints = 0 #@param {type:"slider", min:0, max:10, step:1}
#@markdown * **NOTE**: 0 means "keep all models saved", which could potentially cause out-of-storage issues.
#@markdown * Without training states, each model "only" takes up ~1.6GB. You should have ~50GB of free space to begin with.
#@markdown * With training states, each model (pth+state) takes up ~4.9 GB; Colab will crash around ~10 undeleted checkpoints in this case.

#@markdown ##_Other training parameters_
Fp16 = False #@param {type:"boolean"}
Use8bit = True #@param {type:"boolean"}
#@markdown * **NOTE**: for some reason, fp16 does not seem to improve vram use when combined with 8bit [citation needed]. To be verified later...
TrainingRate = "1e-5" #@param {type:"string"}
TortoiseCompat = False #@param {type:"boolean"}

#@markdown * **NOTE**: TortoiseCompat introduces some breaking changes to the training process. **If you want to reproduce older models**, disable this checkbox.

#@markdown ##_Calculated settings_ override
#@markdown #####Blank entries rely on the calculated defaults from the cell above.
#@markdown ######**Leave them blank unless you want to adjust them manually**
TrainBS = "" #@param {type:"string"}
ValBS = "" #@param {type:"string"}
ValFreq = "" #@param {type:"string"}
LRDecaySteps = "" #@param {type:"string"}
PrintFreq = "" #@param {type:"string"}
SaveCheckpointFreq = "" #@param {type:"string"}

def take(orig, override):
  if override == "": return orig
  return type(orig)(override)

train_bs = take(train_bs, TrainBS)
val_bs = take(val_bs, ValBS)
val_freq = take(val_freq, ValFreq)
lr_decay_steps = eval(LRDecaySteps) if LRDecaySteps else lr_decay_steps
print_freq = take(print_freq, PrintFreq)
save_checkpoint_freq = take(save_checkpoint_freq, SaveCheckpointFreq)
assert len(lr_decay_steps) == 4
gen_lr_steps = ', '.join(str(v) for v in lr_decay_steps)

#@markdown #Run this cell after you finish editing the settings.

## 7. Applying settings:
The code applies the defined settings to a fresh YAML configuration file using `sed` commands.

In [ ]:
%cd /content/DL-Art-School
# !wget https://raw.githubusercontent.com/152334H/DL-Art-School/master/experiments/EXAMPLE_gpt.yml -O experiments/EXAMPLE_gpt.yml
!wget https://raw.githubusercontent.com/josuebatista/DL-Art-School/master/experiments/EXAMPLE_gpt.yml -O experiments/EXAMPLE_gpt.yml

import os
%cd /content/DL-Art-School
!sed -i 's/batch_size: 128/batch_size: '"$train_bs"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/batch_size: 64/batch_size: '"$val_bs"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/val_freq: 500/val_freq: '"$val_freq"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/500, 1000, 1400, 1800/'"$gen_lr_steps"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/print_freq: 100/print_freq: '"$print_freq"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/save_checkpoint_freq: 500/save_checkpoint_freq: '"$save_checkpoint_freq"'/g' ./experiments/EXAMPLE_gpt.yml

!sed -i 's+CHANGEME_validation_dataset_name+'"$ValidationDataset_Name"'+g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's+CHANGEME_path_to_validation_dataset+'"$ValidationDataset_Training_Path"'+g' ./experiments/EXAMPLE_gpt.yml
if(Fp16==True):
  os.system("sed -i 's+fp16: false+fp16: true+g' ./experiments/EXAMPLE_gpt.yml")
!sed -i 's/use_8bit: true/use_8bit: '"$Use8bit"'/g' ./experiments/EXAMPLE_gpt.yml

!sed -i 's/disable_state_saving: true/disable_state_saving: '"$SaveTrainingStates"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/tortoise_compat: True/tortoise_compat: '"$TortoiseCompat"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/number_of_checkpoints_to_save: 0/number_of_checkpoints_to_save: '"$Keep_Last_N_Checkpoints"'/g' ./experiments/EXAMPLE_gpt.yml


!sed -i 's/CHANGEME_training_dataset_name/'"$Dataset_Training_Name"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's/CHANGEME_your_experiment_name/'"$Experiment_Name"'/g' ./experiments/EXAMPLE_gpt.yml
!sed -i 's+CHANGEME_path_to_training_dataset+'"$Dataset_Training_Path"'+g' ./experiments/EXAMPLE_gpt.yml


if (not TrainingRate=="1e-5"):
  os.system("sed -i 's+!!float 1e-5 # CHANGEME:+!!float '" + TrainingRate + "' #+g' ./experiments/EXAMPLE_gpt.yml")

# Print the contents of the file
with open('./experiments/EXAMPLE_gpt.yml', 'r') as configfile:
    print(configfile.read())

## 8. Training:
Finally, the code starts the training process by running the `train.py` script with the configured YAML file.

Press the stop button for this cell when you are satisfied with the results, and have seen:

`INFO:base:Saving models and training states.`

If your training run saves many models, you might exceed the storage limits on the colab runtime. To prevent this, try to delete old checkpoints in /content/DL-Art-School/experiments/$Experiment_Name/(models|training_state)/* via the file explorer panel as the training runs. **Resuming training after a crash requires config editing,** so try to not let that happen.


<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter09/images/ch09_3_Google_Colab_DLAS_checkpoint_directory.png">


In [9]:
%cd /content/DL-Art-School/codes

!python3 train.py -opt ../experiments/EXAMPLE_gpt.yml

/content/DL-Art-School/codes
Disabled distributed training.
24-04-11 23:21:05.167 - INFO:   name: Learn_OAI_Whisper_20240411_JRB
  model: extensibletrainer
  scale: 1
  gpu_ids: [0]
  start_step: 0
  checkpointing_enabled: True
  fp16: False
  use_8bit: True
  wandb: False
  use_tb_logger: True
  datasets:[
    train:[
      name: TestDataset
      n_workers: 8
      batch_size: 4
      mode: paired_voice_audio
      path: /content/gdrive/MyDrive/output/Learn_OAI_Whisper_Sample_Audio01.mp3_2024_04_11-21_39/train.txt
      fetcher_mode: ['lj']
      phase: train
      max_wav_length: 255995
      max_text_length: 200
      sample_rate: 22050
      load_conditioning: True
      num_conditioning_candidates: 2
      conditioning_length: 44000
      use_bpe_tokenizer: True
      load_aligned_codes: False
      data_type: img
    ]
    val:[
      name: TestValidation
      n_workers: 1
      batch_size: 1
      mode: paired_voice_audio
      path: /content/gdrive/MyDrive/output/Learn_OAI_Wh

## 9. Exporting to Google Drive:
After training, the code provides an option to copy the experiment folder to the user's Google Drive for persistence.

In [10]:
!cp -r /content/DL-Art-School/experiments/$Experiment_Name /content/gdrive/MyDrive/

After running the cell, go to your Google Drive using a web browser, and you will see a directory with the same name of the value of the `Experiment_Name` variable.

<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter09/images/ch09_3_Google_drive_DLAS_checkpoint_directory.JPG">

Inside the `<Experiment_Name>/models` folder you will find the model checkpoints, they are the files with extension `.pth`

<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter09/images/ch09_3_Google_Colab_DLAS_checkpoint_pth_file.JPG">

That `.pth` file is the fine-tuned voice model we just cloned. In the next step, we will use that file to sythetize the voice using `TorToiSe-tts-fast`.